In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# User-Agent para simular una solicitud desde un navegador
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

# Lista para almacenar los datos extraídos de todas las páginas
data = []

# Función para procesar una página de productos
def procesar_pagina(categoria_url):
    # Realizar la solicitud HTTP con headers
    response = requests.get(categoria_url, headers=headers)

    # Verificar que la solicitud fue exitosa
    if response.status_code == 200:
        # Analizar el HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontrar todos los elementos de productos (contenidos en un 'li' con clase 'product')
        products = soup.find_all('li', class_='product')

        if not products:
            print("No se encontraron productos en esta categoría.")
        else:
            for product in products:
                try:
                    # Extraer el nombre del producto
                    name = product.find('h2', class_='woocommerce-loop-product__title').text.strip()

                    # Extraer el precio con descuento (dentro de <ins>) si está disponible
                    discounted_price_tag = product.find('ins')
                    if discounted_price_tag:
                        discounted_price = discounted_price_tag.find('bdi').text.strip()
                    else:
                        discounted_price = "No disponible"

                    # Extraer el precio original (dentro de <del>) si está disponible
                    original_price_tag = product.find('del')
                    if original_price_tag:
                        original_price = original_price_tag.find('bdi').text.strip()
                    else:
                        # Si no hay precio original, el precio actual se encuentra dentro de <span>
                        current_price_tag = product.find('span', class_='woocommerce-Price-amount amount')
                        if current_price_tag:
                            original_price = current_price_tag.find('bdi').text.strip()
                        else:
                            original_price = "No disponible"

                    # Extraer el enlace del producto
                    link = product.find('a')['href']

                    # Imprimir resultados para ver qué está sucediendo en cada paso
                    print(f"Producto: {name}")
                    print(f"Precio Original: {original_price}")
                    print(f"Precio con Descuento: {discounted_price}")
                    print(f"Link: {link}")
                    print("-" * 50)

                    # Almacenar datos en la lista
                    data.append({
                        'Nombre': name,
                        'Precio Original': original_price,
                        'Precio con Descuento': discounted_price,
                        'Link': link
                    })
                except AttributeError as e:
                    print(f"Error procesando el producto: {e}")
                    continue
    else:
        print(f"Error al acceder a la página: {response.status_code}")

# Procesar las primeras 10 páginas de la categoría
for page in range(1, 11):
    url = f"https://lamartine.cl/productos/page/{page}/"
    print(f"Procesando página {page}...")
    procesar_pagina(url)

# Crear un DataFrame con los datos
df = pd.DataFrame(data)

# Verificar cuántos productos se encontraron en total
print(f"Se encontraron {len(df)} productos en total.")

# Guardar en un archivo CSV si se encuentran productos
if len(df) > 0:
    df.to_csv('repuestos_lamartine_productos.csv', index=False)
    print("Datos guardados en 'repuestos_lamartine_productos.csv'")
else:
    print("No se encontraron productos suficientes.")








Procesando página 1...
Producto: ACEITE  ATF+6 DEXRON VI  ACDELCO
Precio Original: $16,000
Precio con Descuento: $12,000
Link: https://lamartine.cl/producto/aceite-atf6-acdelco-dexron-vi-acdelco/
--------------------------------------------------
Producto: ACEITE  CAJA  TRANSFERENCIA  AUTO-TRACK II  (1 LITRO)  ACDELCO
Precio Original: $16,000
Precio con Descuento: $12,000
Link: https://lamartine.cl/producto/aceite-de-caja-de-transferencia-946ml-auto-track-ii-acdelco/
--------------------------------------------------
Producto: ACEITE 0W20 SINTETICO PENNZOIL DEXOS 1 946ml
Precio Original: $22,000
Precio con Descuento: $16,500
Link: https://lamartine.cl/producto/aceite-0w20-sintetico-pennzoil-dexos-1-mopar/
--------------------------------------------------
Producto: ACEITE 0W20 SINTÉTICO WILLIAMS DEXOS 1 946ml USA
Precio Original: $13,000
Precio con Descuento: $9,750
Link: https://lamartine.cl/producto/aceite-0w20-sintetico-williams-dexos-1-946ml-usa/
-----------------------------------

In [2]:
df

,Nombre,Precio Original,Precio con Descuento,Link
0,ACEITE ATF+6 DEXRON VI ACDELCO,"$16,000","$12,000",https://lamartine.cl/producto/aceite-atf6-acde...
1,ACEITE CAJA TRANSFERENCIA AUTO-TRACK II (1...,"$16,000","$12,000",https://lamartine.cl/producto/aceite-de-caja-d...
2,ACEITE 0W20 SINTETICO PENNZOIL DEXOS 1 946ml,"$22,000","$16,500",https://lamartine.cl/producto/aceite-0w20-sint...
3,ACEITE 0W20 SINTÉTICO WILLIAMS DEXOS 1 946ml USA,"$13,000","$9,750",https://lamartine.cl/producto/aceite-0w20-sint...
4,ACEITE 0W40 SINTETICO MOTORES SRT 946 ml PENN...,"$19,200","$14,400",https://lamartine.cl/producto/aceite-0w40-sint...
...,...,...,...,...
115,AMORTIGUADOR CAPOT CHEVROLET TRAVERSE 09-16 AMS,"$36,000","$27,000",https://lamartine.cl/producto/amortiguador-cap...
116,AMORTIGUADOR CAPOT DERECHO F150 15-19 AMS,"$36,000","$27,000",https://lamartine.cl/producto/amortiguador-cap...
117,AMORTIGUADOR CAPOT DODGE RAM 1500/3500 2004-20...,"$28,000","$21,000",https://lamartine.cl/producto/amortiguador-cap...
118,AMORTIGUADOR CAPOT EQUINOX 10-16 AMS,"$39,000","$29,250",https://lamartine.cl/producto/amortiguador-cap...
